In [ ]:
# Core
import sys
import pandas as pd
import numpy as np
from pathlib import Path

## Plotting
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

## Harp/Bonsai
sys.path.append('../../src/')
from bonsai import load_bonsai_config
load_bonsai_config(r"C:\git\AllenNeuralDynamics\aind-vr-foraging\Bonsai")
import harp
import harp.processing
import data_io

#Global Viz settings
sns.set_style('darkgrid') # darkgrid, white grid, dark, white and ticks
plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=14)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=13)    # fontsize of the tick labels
plt.rc('ytick', labelsize=13)    # fontsize of the tick labels
plt.rc('legend', fontsize=13)    # legend fontsize
plt.rc('font', size=13)          # controls default text sizes

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['font.family'] = 'Arial'

default_img_size = (15, 8)

In [ ]:
session_path = Path(r"Z:\scratch\vr-foraging\672107\20231013T111657")


# Harp Devices:
HarpBehavior = harp.HarpDevice("Behavior")
harp_behavior_data = data_io.HarpSource(device=HarpBehavior, path=session_path / "Behavior", name="behavior", autoload=False)

software_events = data_io.SoftwareEventSource(path=session_path / "SoftwareEvents", name="software_events", autoload=True)
config = data_io.ConfigSource(path=session_path / "Config", name="config", autoload=True)
operation_control = data_io.OperationControlSource(path=session_path / "OperationControl", name="config", autoload=False)

def add_position(df: pd.DataFrame, position: data_io.DataStream):

    df = pd.merge_asof(df.sort_index(), position.data.sort_index(), direction='nearest', on="Seconds").set_index("Seconds").sort_index()
    df.columns = [*df.columns[:-1], 'Position']
    return df

In [ ]:
## Find changes in reward available
patches = software_events.streams.ActivePatch.data
reward = software_events.streams.GiveReward.data
reward_available = patches.iloc[0]["data"]["patchRewardFunction"]["initialRewardAmount"]

reward_updates = pd.concat([patches, reward])
reward_updates.sort_index(inplace=True)
reward_updates["reward_in_site"] = np.nan
reward_updates["no_reward_in_patch_count"] = np.nan
for event in reward_updates.iterrows():
    if event[1]["name"] == 'GiveReward': #update reward
        reward_available -= event[1]["data"]
    elif event[1]["name"] == 'ActivePatch': #reset reward
        reward_available = event[1]["data"]["patchRewardFunction"]["initialRewardAmount"]
    else:
        raise ValueError("Unknown event type")
    reward_updates.at[event[0], "reward_in_site"] = reward_available



In [ ]:
# Find responses to Reward site
activeSite = software_events.streams.ActiveSite.data
choiceFeedback = software_events.streams.ChoiceFeedback.data
OdorSites = activeSite[activeSite["data"].apply(lambda x : x['label']) == 'Reward']
OdorSites["patch_number"] = -1
OdorSites["patch_label"] = ''

OdorSites["site_number"] = -1
OdorSites["is_choice"] = False
OdorSites["is_reward"] = 0
OdorSites["past_no_reward_count"] = 0
past_no_reward_counter = 0
current_patch_idx = -1

site_number = 0
for idx, event in enumerate(OdorSites.iterrows()):
    arg_min, val_min = harp.processing.find_closest(event[0], patches.index.values, mode="below_zero")
    if not(np.isnan(arg_min)):
        OdorSites.loc[event[0], "patch_number"] = arg_min
    if current_patch_idx != arg_min:
        current_patch_idx = arg_min
        site_number = 0
    else:
        site_number += 1
    OdorSites.loc[event[0], "site_number"] = site_number

    if idx < len(OdorSites) -1:
        choice = choiceFeedback.loc[(choiceFeedback.index >= OdorSites.index[idx]) & (choiceFeedback.index < OdorSites.index[idx+1])]
        reward_in_site = reward.loc[(reward.index >= OdorSites.index[idx]) & (reward.index < OdorSites.index[idx+1])]
    else:
        choice = choiceFeedback.loc[(choiceFeedback.index >= OdorSites.index[idx])]
        reward_in_site = reward.loc[(reward.index >= OdorSites.index[idx])]
    OdorSites.loc[event[0], "is_choice"] = len(choice) > 0
    OdorSites.loc[event[0], "is_reward"] = reward_in_site.iloc[0]["data"] if len(reward_in_site) > 0 else np.nan
    OdorSites.loc[event[0], "past_no_reward_count"] = past_no_reward_counter
    if OdorSites.loc[event[0], "is_reward"] == 0:
        past_no_reward_counter += 1
    else:
        past_no_reward_counter = 0

OdorSites["patch_label"] = OdorSites["patch_number"].apply(lambda x : patches.iloc[x]["data"]["label"])


In [ ]:
OdorSites["is_reward"].plot()
OdorSites["past_no_reward_count"].plot()

In [ ]:
## P(Stay) vs Visit Number

session_duration = OdorSites.index[-1] - OdorSites.index[0]
threshold = OdorSites.index[0] + session_duration * 1
for patch_type, patch_type_df in OdorSites[OdorSites.index < threshold].groupby("patch_label"):
    fig, ax = plt.subplots(1, 1, figsize=np.array(default_img_size)/2)
    choice_probability = np.zeros(len(patch_type_df["site_number"].unique()))
    n_choices = np.zeros_like(choice_probability)
    for site_number, site_number_df in patch_type_df.groupby("site_number"):
        choice_probability[site_number] = site_number_df["is_choice"].sum() / len(site_number_df)
        n_choices[site_number] = len(site_number_df)
    ax.set_title(patch_type)
    ax.plot(np.arange(0, choice_probability.shape[0]), choice_probability, color = 'k', label="P(Stay)")
    ax.set_xlabel("site_number")
    ax.set_ylabel("P(Stay)")
    ax.set_ylim([-0.05, 1.05])
    ax2 = ax.twinx()
    ax2.plot(np.arange(0, choice_probability.shape[0]), n_choices, color = 'r', label="Number of trials")
    ax2.set_ylabel("n_trials")
    ax2.set_ylim(bottom=0)
    ax2.grid(False)


In [ ]:
## P(Leave) vs Number of NO past rewards

session_duration = OdorSites.index[-1] - OdorSites.index[0]
threshold = OdorSites.index[0] + session_duration * 1
for patch_type, patch_type_df in OdorSites[OdorSites.index < threshold].groupby("patch_label"):
    fig, ax = plt.subplots(1, 1, figsize=np.array(default_img_size)/2)
    choice_probability = np.zeros(len(patch_type_df["past_no_reward_count"].unique()))
    n_trials = np.zeros_like(choice_probability)
    for n_no_rewards, no_reward_df in patch_type_df.groupby("past_no_reward_count"):
        choice_probability[n_no_rewards] = no_reward_df["is_choice"].sum() / len(no_reward_df)
        n_trials[n_no_rewards] = len(no_reward_df)
    ax.set_title(patch_type)
    ax.plot(np.arange(0, choice_probability.shape[0]), choice_probability, color = 'k', label="P(Stay)")
    ax.set_xlabel("n_no_rewards")
    ax.set_ylabel("P(Stay)")
    ax.set_ylim([-0.05, 1.05])
    ax2 = ax.twinx()
    ax2.plot(np.arange(0, choice_probability.shape[0]), n_trials, color = 'r', label="Number of trials")
    ax2.set_ylabel("n_trials")
    ax2.set_ylim(bottom=0)
    ax2.grid(False)